In [19]:
import pandas as pd
import csv
import json

In [20]:
Temp_change=pd.read_csv('D:/Ruchita/NCI/sem1/DAP/DATASET1/Temperature_Change.csv')
Temp_change.head()

,Country,Date,Temperature
0,Aruba,1961-01-01,-0.232
1,Aruba,1962-01-01,0.072
2,Aruba,1963-01-01,-0.041
3,Aruba,1964-01-01,0.118
4,Aruba,1965-01-01,-0.237


In [21]:
with open("D:/Ruchita/NCI/sem1/DAP/DATASET1/Temperature_Change.csv","r") as f:
    reader= csv.reader(f)
    next(reader)
    Temp=[]
    for row in reader:
        Temp.append({"Country":row[0],"Date":row[1],"Temperature":row[2]})

In [22]:
with open("Finalfile.json","w") as f:
    json.dump(Temp, f, indent=4)


In [23]:
Temp

[{'Country': 'Aruba',
  'Date': '1961-01-01',
  'Temperature': '-0.23199999999999998'},
 {'Country': 'Aruba',
  'Date': '1962-01-01',
  'Temperature': '0.07200000000000001'},
 {'Country': 'Aruba',
  'Date': '1963-01-01',
  'Temperature': '-0.040999999999999995'},
 {'Country': 'Aruba',
  'Date': '1964-01-01',
  'Temperature': '0.11800000000000001'},
 {'Country': 'Aruba', 'Date': '1965-01-01', 'Temperature': '-0.237'},
 {'Country': 'Aruba', 'Date': '1966-01-01', 'Temperature': '0.05'},
 {'Country': 'Aruba', 'Date': '1967-01-01', 'Temperature': '-0.326'},
 {'Country': 'Aruba', 'Date': '1968-01-01', 'Temperature': '-0.132'},
 {'Country': 'Aruba', 'Date': '1969-01-01', 'Temperature': '0.423'},
 {'Country': 'Aruba', 'Date': '1970-01-01', 'Temperature': '0.182'},
 {'Country': 'Aruba', 'Date': '1971-01-01', 'Temperature': '-0.375'},
 {'Country': 'Aruba', 'Date': '1972-01-01', 'Temperature': '-0.075'},
 {'Country': 'Aruba', 'Date': '1973-01-01', 'Temperature': '0.141'},
 {'Country': 'Aruba', 'D

In [24]:
#To connect mongodb 
from pymongo import MongoClient
import pymongo as pym

client = MongoClient('127.0.0.1', 27017)
db = client['Temperature']
Temperature_Change = db['Temperature_C']
result = Temperature_Change.insert_many(Temp)


In [25]:
Temp_cursor = Temperature_Change.find()
Temp_datalist = list(Temp_cursor)
Temp_df = pd.DataFrame(Temp_datalist)

In [27]:
Temp_df.head()

,_id,Country,Date,Temperature
0,5ff7b87e955e67805baeab23,Aruba,1961-01-01,-0.23199999999999998
1,5ff7b87e955e67805baeab24,Aruba,1962-01-01,0.07200000000000001
2,5ff7b87e955e67805baeab25,Aruba,1963-01-01,-0.040999999999999995
3,5ff7b87e955e67805baeab26,Aruba,1964-01-01,0.11800000000000001
4,5ff7b87e955e67805baeab27,Aruba,1965-01-01,-0.237


In [28]:
Temp_df1=Temp_df.drop(['_id'],axis=1)
Temp_df1


,Country,Date,Temperature
0,Aruba,1961-01-01,-0.23199999999999998
1,Aruba,1962-01-01,0.07200000000000001
2,Aruba,1963-01-01,-0.040999999999999995
3,Aruba,1964-01-01,0.11800000000000001
4,Aruba,1965-01-01,-0.237
...,...,...,...
9828,Zimbabwe,2015-01-01,1.091
9829,Zimbabwe,2016-01-01,1.3030000000000002
9830,Zimbabwe,2017-01-01,0.26899999999999996
9831,Zimbabwe,2018-01-01,0.496


In [29]:
Temp_df1.to_csv('M_Temperature.csv', index = None)

In [ ]:
import psycopg2
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "postgres")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute('CREATE DATABASE Global_Warming;')
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [30]:
createString = """
CREATE TABLE Temperature_Change1(
    Country varchar(255),
    Date varchar(255),
    Temperature varchar(255)
);
"""
try: 
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "population")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute(createString)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL", dbError)
finally:
    if(dbConnection): dbConnection.close()


In [31]:
import csv
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "population")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    insertString = "INSERT INTO Temperature_Change1 VALUES ('{}',"+"'{}',"+"'{}')"
    with open('D:/Ruchita/NCI/sem1/DAP/DATASET1/M_Temperature.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader) # skip the header
        for row in reader:
            dbCursor.execute(insertString.format(*row))
    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [32]:
import pandas.io.sql as sqlio
query = """ SELECT *  FROM Temperature_Change1;"""
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "192.168.56.30",
        port = "5432",
        database = "population")
    Temperature_change = sqlio.read_sql_query(query, dbConnection) 
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if(dbConnection): dbConnection.close()

In [33]:
Temperature_change

,country,date,temperature
0,Aruba,1961-01-01,-0.23199999999999998
1,Aruba,1962-01-01,0.07200000000000001
2,Aruba,1963-01-01,-0.040999999999999995
3,Aruba,1964-01-01,0.11800000000000001
4,Aruba,1965-01-01,-0.237
...,...,...,...
9828,Zimbabwe,2015-01-01,1.091
9829,Zimbabwe,2016-01-01,1.3030000000000002
9830,Zimbabwe,2017-01-01,0.26899999999999996
9831,Zimbabwe,2018-01-01,0.496
